In [ ]:
# Flattenig data from json 
import json
import pandas as pd

zipcodes_datafile_geojson = '/home/dvieira/tmp/easyhealth/eh-analytics/src/2023-05-19_01.58.58-usa_zip_codes_v7.geo.json'

with open(zipcodes_datafile_geojson) as file:
    data = json.load(file)

rows = []
for feature in data["features"]:
    row = {
        "zip": feature["properties"]["zip"],
        "type": feature["geometry"]["type"],
        "coordinates": feature["geometry"]["coordinates"]
    }
    rows.append(row)

df = pd.DataFrame(rows)
print(df)


In [ ]:
from sqlalchemy import Column, Integer, Sequence, String, create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm.session import Session

# eng = create_engine("duckdb:///:memory:")
eng = create_engine("duckdb:///database.duckdb")

df.to_sql('draft', eng, if_exists='replace')




In [ ]:
with eng.connect() as connection:
    result = connection.execute(text("SELECT * FROM draft"))  # Using text() for SQL statement

    for i, row in enumerate(result):
        if i >= 10:
            break
        
        print(row)

    connection.close()


In [ ]:
df_2 = pd.read_sql('draft', eng)
print(df_2.head(10))



In [ ]:
# with eng.connect() as connection: 
#     result = connection.execute(text("SELECT * FROM draft"))
#     rows = result.fetchall()

# for row in rows:
#     print(row)


In [ ]:
# connection.close()

In [ ]:
print(df.dtypes)

In [ ]:
import duckdb

duckdb.default_connection.execute("SET GLOBAL pandas_analyze_sample=100000")

con = duckdb.connect('database.duckdb')

con.register('draft', df)

con.query('SELECT * FROM draft')

# con.fetchall()

In [ ]:
# Configurar o python to throw data into clickhouse 
# pip install clickhouse-connect

import pandas as pd
import clickhouse_connect

# Assuming you have a pandas DataFrame named df with your data
df = pd.DataFrame({
    'key': [1000, 2000],
    'value': ['String Value 1000', 'String Value 2000'],
    'metric': [5.233, -107.04]
})

# connect to ClickHouse server on localhost
client = clickhouse_connect.get_client(host='localhost', username='username', password='password')

# If using ClickHouse Cloud service, use this connection string
# client = clickhouse_connect.get_client(host='HOSTNAME.clickhouse.cloud', port=8443, username='default', password='your password')

# Create the table in ClickHouse if it doesn't exist
client.command('CREATE TABLE IF NOT EXISTS new_table (key UInt32, value String, metric Float64) ENGINE = MergeTree() ORDER BY key')

# Convert DataFrame to a list of tuples, which is what ClickHouse Connect expects
data_tuples = list(df.itertuples(index=False, name=None))

# Define the column names of your dataframe, matching the table schema
column_names = ['key', 'value', 'metric']

# insert DataFrame records one by one in clickhouse table
client.insert('new_table', data_tuples, column_names=column_names)

# retrieve data from ClickHouse
result = client.query('SELECT max(key), avg(metric) FROM new_table')
print(result.result_rows)



In [ ]:
#This code get geoson file with zipcodes from US and inserts it into a clickhouse table.
# It works #

import json
import pandas as pd
import clickhouse_connect

zipcodes_datafile_geojson = '/home/dvieira/tmp/easyhealth/eh-analytics/src/2023-05-19_01.58.58-usa_zip_codes_v7.geo.json'

# Load data from JSON
with open(zipcodes_datafile_geojson) as file:
    data = json.load(file)

rows = []
for feature in data["features"]:
    row = {
        "zip": feature["properties"]["zip"],
        "type": feature["geometry"]["type"],
        "coordinates": str(feature["geometry"]["coordinates"])
    }
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)
print(df)

# Establish connection
client = clickhouse_connect.get_client(host='localhost', username='username', password='password')

# Create table
client.command('''
    CREATE TABLE IF NOT EXISTS zip_codes (
        zip String,
        type String,
        coordinates String
    ) ENGINE = MergeTree() ORDER BY zip
''')

# Prepare data for insertion
data = [list(row) for row in df.itertuples(index=False, name=None)]

# Insert data
client.insert('zip_codes', data, column_names=['zip', 'type', 'coordinates'])

In [ ]:
import pandas as pd

# Define the field mapping dictionary with readable names and original descriptions
field_mapping =   {
    'STATEFIPS':  {'readable_name': 'state_fips_code', 'description': 'The State Federal Information Processing System (FIPS) code'},
    'STATE':      {'readable_name': 'state', 'description': 'The State associated with the ZIP code'},
    'ZIPCODE':    {'readable_name': 'zip_code', 'description': '5-digit Zip code'},
    'AGI_STUB':   {'readable_name': 'size_of_adjusted_gross_income', 'description': 'Size of adjusted gross income'},
    'N1':         {'readable_name': 'num_returns', 'description': 'Number of returns'},
    'MARS1':      {'readable_name': 'num_single_returns', 'description': 'Number of single returns'},
    'MARS2':      {'readable_name': 'num_joint_returns', 'description': 'Number of joint returns'},
    'MARS4':      {'readable_name': 'num_head_of_household_returns', 'description': 'Number of head of household returns'},
    'ELF':        {'readable_name': 'num_electronically_filed_returns', 'description': 'Number of electronically filed returns'},
    'CPREP':      {'readable_name': 'num_computer_prepared_paper_returns', 'description': 'Number of computer prepared paper returns'},
    'PREP':       {'readable_name': 'num_returns_with_paid_preparers_signature', 'description': "Number of returns with paid preparer's signature"},
    'DIR_DEP':    {'readable_name': 'num_returns_with_direct_deposit', 'description': 'Number of returns with direct deposit'},
    'VRTCRIND':   {'readable_name': 'num_returns_with_virtual_currency_indicator', 'description': 'Number of returns with virtual currency indicator'},
    'N2':         {'readable_name': 'num_individuals', 'description': 'Number of individuals'},
    'TOTAL_VITA': {'readable_name': 'total_num_volunteer_prepared_returns', 'description': 'Total number of volunteer prepared returns'},
    'VITA':       {'readable_name': 'num_vita_prepared_returns', 'description': 'Number of volunteer income tax assistance (VITA) prepared returns'},
    'TCE':        {'readable_name': 'num_tce_prepared_returns', 'description': 'Number of tax counseling for the elderly (TCE) prepared returns'},
    'VITA_EIC':   {'readable_name': 'num_volunteer_prepared_returns_with_earned_income_credit', 'description': 'Number of volunteer prepared returns with Earned Income Credit'},
    'RAC':        {'readable_name': 'num_refund_anticipation_check_returns', 'description': 'Number of refund anticipation check returns'},
    'ELDERLY':    {'readable_name': 'num_elderly_returns', 'description': 'Number of elderly returns'},
    'A00100':     {'readable_name': 'adjusted_gross_income', 'description': 'Adjust gross income (AGI)'},
    'N02650':     {'readable_name': 'num_returns_with_total_income', 'description': 'Number of returns with total income'},
    'A02650':     {'readable_name': 'total_income_amount', 'description': 'Total income amount'},
    'N00200':     {'readable_name': 'num_returns_with_salaries_wages', 'description': 'Number of returns with salaries and wages'},
    'A00200':     {'readable_name': 'salaries_wages_amount', 'description': 'Salaries and wages amount'},
    'N00300':     {'readable_name': 'num_returns_with_taxable_interest', 'description': 'Number of returns with taxable interest'},
    'A00300':     {'readable_name': 'taxable_interest_amount', 'description': 'Taxable interest amount'},
    'N00600':     {'readable_name': 'num_returns_with_ordinary_dividends', 'description': 'Number of returns with ordinary dividends'},
    'A00600':     {'readable_name': 'ordinary_dividends_amount', 'description': 'Ordinary dividends amount'},
    'N00650':     {'readable_name': 'num_returns_with_qualified_dividends', 'description': 'Number of returns with qualified dividends'},
    'A00650':     {'readable_name': 'qualified_dividends_amount', 'description': 'Qualified dividends amount'},
    'N00700':     {'readable_name': 'num_returns_with_state_local_income_tax_refunds', 'description': 'Number of returns with state and local income tax refunds'},
    'A00700':     {'readable_name': 'state_local_income_tax_refunds_amount', 'description': 'State and local income tax refunds amount'},
    'N00900':     {'readable_name': 'num_returns_with_business_professional_net_income', 'description': 'Number of returns with business or professional net income (less loss)'},
    'A00900':     {'readable_name': 'business_professional_net_income_amount', 'description': 'Business or professional net income (less loss) amount'},
    'N01000':     {'readable_name': 'num_returns_with_net_capital_gain', 'description': 'Number of returns with net capital gain (less loss)'},
    'A01000':     {'readable_name': 'net_capital_gain_amount', 'description': 'Net capital gain (less loss) amount'},
    'N01400':     {'readable_name': 'num_returns_with_taxable_IRA_distributions', 'description': 'Number of returns with taxable individual retirement arrangements distributions'},
    'A01400':     {'readable_name': 'taxable_IRA_distributions_amount', 'description': 'Taxable individual retirement arrangements distributions amount'},
    'N01700':     {'readable_name': 'num_returns_with_taxable_pensions_annuities', 'description': 'Number of returns with taxable pensions and annuities'},
    'A01700':     {'readable_name': 'taxable_pensions_annuities_amount', 'description': 'Taxable pensions and annuities amount'},
    'SCHF':       {'readable_name': 'num_farm_returns', 'description': 'Number of farm returns'},
    'N02300':     {'readable_name': 'num_returns_with_unemployment_compensation', 'description': 'Number of returns with unemployment compensation'},
    'A02300':     {'readable_name': 'unemployment_compensation_amount', 'description': 'Unemployment compensation amount'},
    'N02500':     {'readable_name': 'num_returns_with_taxable_Social_Security_benefits', 'description': 'Number of returns with taxable Social Security benefits'},
    'A02500':     {'readable_name': 'taxable_Social_Security_benefits_amount', 'description': 'Taxable Social Security benefits amount'},
    'N26270':     {'readable_name': 'num_returns_with_partnership_S-corp_net_income', 'description': 'Number of returns with partnership/S-corp net income (less loss)'},
    'A26270':     {'readable_name': 'partnership_S-corp_net_income_amount', 'description': 'Partnership/S-corp net income (less loss) amount'},
    'N02900':     {'readable_name': 'num_returns_with_total_statutory_adjustments', 'description': 'Number of returns with total statutory adjustments'},
    'A02900':     {'readable_name': 'total_statutory_adjustments_amount', 'description': 'Total statutory adjustments amount'},
    'N03220':     {'readable_name': 'num_returns_with_educator_expenses', 'description': 'Number of returns with educator expenses'},
    'A03220':     {'readable_name': 'educator_expenses_amount', 'description': 'Educator expenses amount'},
    'N03300':     {'readable_name': 'num_returns_with_self_employed_keogh_retirement_plans', 'description': 'Number of returns with Self-employed (Keogh) retirement plans'},
    'A03300':     {'readable_name': 'self_employed_keogh_retirement_plans_amount', 'description': 'Self-employed (Keogh) retirement plans amount'},
    'N03270':     {'readable_name': 'num_returns_with_self_employed_health_insurance_deduction', 'description': 'Number of returns with Self-employed health insurance deduction'},
    'A03270':     {'readable_name': 'self_employed_health_insurance_deduction_amount', 'description': 'Self-employed health insurance deduction amount'},
    'N03150':     {'readable_name': 'num_returns_with_individual_retirement_arrangement_payments', 'description': 'Number of returns with Individual retirement arrangement payments'},
    'A03150':     {'readable_name': 'individual_retirement_arrangement_payments_amount', 'description': 'Individual retirement arrangement payments amount'},
    'N03210':     {'readable_name': 'num_returns_with_student_loan_interest_deduction', 'description': 'Number of returns with student loan interest deduction'},
    'A03210':     {'readable_name': 'student_loan_interest_deduction_amount', 'description': 'Student loan interest deduction amount'},
    'N02910':     {'readable_name': 'num_returns_with_charitable_contributions_if_took_standard_deduction', 'description': 'Number of returns with charitable contributions if took standard deduction'},
    'A02910':     {'readable_name': 'charitable_contributions_if_took_standard_deduction', 'description': 'Charitable contributions if took standard deduction'},
    'N04450':     {'readable_name': 'num_returns_with_total_standard_deduction', 'description': 'Number of returns with total standard deduction'},
    'A04450':     {'readable_name': 'total_standard_deduction_amount', 'description': 'Total standard deduction amount'},
    'N04100':     {'readable_name': 'num_returns_with_basic_standard_deduction', 'description': 'Number of returns with basic standard deduction'},
    'A04100':     {'readable_name': 'basic_standard_deduction_amount', 'description': 'Basic standard deduction amount'},
    'N04200':     {'readable_name': 'num_returns_with_additional_standard_deduction', 'description': 'Number of returns with additional standard deduction'},
    'A04200':     {'readable_name': 'additional_standard_deduction_amount', 'description': 'Additional standard deduction amount'},
    'N04470':     {'readable_name': 'num_returns_with_itemized_deductions', 'description': 'Number of returns with itemized deductions'},
    'A04470':     {'readable_name': 'total_itemized_deductions_amount', 'description': 'Total itemized deductions amount'},
    'A00101':     {'readable_name': 'AGI_for_itemized_returns_amount', 'description': 'Amount of AGI for itemized returns'},
    'N17000':     {'readable_name': 'num_returns_with_total_medical_and_dental_expense_deduction', 'description': 'Number of returns with Total medical and dental expense deduction'},
    'A17000':     {'readable_name': 'total_medical_and_dental_expense_deduction_amount', 'description': 'Total medical and dental expense deduction amount'},
    'N18425':     {'readable_name': 'num_returns_with_state_and_local_income_taxes', 'description': 'Number of returns with State and local income taxes'},
    'A18425':     {'readable_name': 'state_and_local_income_taxes_amount', 'description': 'State and local income taxes amount'},
    'N18450':     {'readable_name': 'num_returns_with_state_and_local_general_sales_tax', 'description': 'Number of returns with State and local general sales tax'},
    'A18450':     {'readable_name': 'state_and_local_general_sales_tax_amount', 'description': 'State and local general sales tax amount'},
    'N18500':     {'readable_name': 'num_returns_with_real_estate_taxes', 'description': 'Number of returns with real estate taxes'},
    'A18500':     {'readable_name': 'real_estate_taxes_amount', 'description': 'Real estate taxes amount'},
    'N18800':     {'readable_name': 'num_returns_with_personal_property_taxes', 'description': 'Number of returns with Personal property taxes'},
    'A18800':     {'readable_name': 'personal_property_taxes_amount', 'description': 'Personal property taxes amount'},
    'N18460':     {'readable_name': 'num_returns_with_limited_state_and_local_taxes', 'description': 'Number of returns with Limited state and local taxes'},
    'A18460':     {'readable_name': 'limited_state_and_local_taxes_amount', 'description': 'Limited state and local taxes'},
    'N18300':     {'readable_name': 'num_returns_with_total_taxes_paid', 'description': 'Number of returns with Total taxes paid'},
    'A18300':     {'readable_name': 'total_taxes_paid_amount', 'description': 'Total taxes paid amount'},
    'N19300':     {'readable_name': 'num_returns_with_home_mortgage_interest_paid', 'description': 'Number of returns with Home mortgage interest paid'},
    'A19300':     {'readable_name': 'home_mortgage_interest_paid_amount', 'description': 'Home mortgage interest paid amount'},
    'N19500':     {'readable_name': 'num_returns_with_home_mortgage_from_personal_seller', 'description': 'Number of returns with Home mortgage from personal seller'},
    'A19500':     {'readable_name': 'home_mortgage_from_personal_seller_amount', 'description': 'Home mortgage from personal seller amount'},
    'N19530':     {'readable_name': 'num_returns_with_deductible_points', 'description': 'Number of returns with Deductible points'},
    'A19530':     {'readable_name': 'deductible_points_amount', 'description': 'Deductible points amount'},
    'N19550':     {'readable_name': 'num_returns_with_qualified_mortgage_insurance_premiums', 'description': 'Number of returns with Qualified mortgage insurance premiums'},
    'A19550':     {'readable_name': 'qualified_mortgage_insurance_premiums_amount', 'description': 'Qualified mortgage insurance premiums amount'},
    'N19570':     {'readable_name': 'num_returns_with_investment_interest_paid', 'description': 'Number of returns with Investment interest paid'},
    'A19570':     {'readable_name': 'investment_interest_paid_amount', 'description': 'Investment interest paid amount'},
    'N19700':     {'readable_name': 'num_returns_with_total_charitable_contributions', 'description': 'Number of returns with Total charitable contributions'},
    'A19700':     {'readable_name': 'total_charitable_contributions_amount', 'description': 'Total charitable contributions amount'},
    'N20950':     {'readable_name': 'num_returns_with_other_non_limited_miscellaneous_deductions', 'description': 'Number of returns with Other non-limited miscellaneous deductions'},
    'A20950':     {'readable_name': 'other_non_limited_miscellaneous_deductions_amount', 'description': 'Other non-limited miscellaneous deductions amount'},
    'N04475':     {'readable_name': 'num_returns_with_qualified_business_income_deduction', 'description': 'Number of returns with Qualified business income deduction'},
    'A04475':     {'readable_name': 'qualified_business_income_deduction_amount', 'description': 'Qualified business income deduction'},
    'N04800':     {'readable_name': 'num_returns_with_taxable_income', 'description': 'Number of returns with taxable income'},
    'A04800':     {'readable_name': 'taxable_income_amount', 'description': 'Taxable income amount'},
    'N05800':     {'readable_name': 'num_returns_with_income_tax_before_credits', 'description': 'Number of returns with income tax before credits'},
    'A05800':     {'readable_name': 'income_tax_before_credits_amount', 'description': 'Income tax before credits amount'},
    'N09600':     {'readable_name': 'num_returns_with_alternative_minimum_tax', 'description': 'Number of returns with alternative minimum tax'},
    'A09600':     {'readable_name': 'alternative_minimum_tax_amount', 'description': 'Alternative minimum tax amount'},
    'N05780':     {'readable_name': 'num_returns_with_excess_advance_premium_tax_credit_repayment', 'description': 'Number of returns with excess advance premium tax credit repayment'},
    'A05780':     {'readable_name': 'excess_advance_premium_tax_credit_repayment_amount', 'description': 'Excess advance premium tax credit repayment amount'},
    'N07100':     {'readable_name': 'num_returns_with_total_tax_credits', 'description': 'Number of returns with total tax credits'},
    'A07100':     {'readable_name': 'total_tax_credits_amount', 'description': 'Total tax credits amount'},
    'N07300':     {'readable_name': 'num_returns_with_foreign_tax_credit', 'description': 'Number of returns with foreign tax credit'},
    'A07300':     {'readable_name': 'foreign_tax_credit_amount', 'description': 'Foreign tax credit amount'},
    'N07180':     {'readable_name': 'num_returns_with_child_and_dependent_care_credit', 'description': 'Number of returns with child and dependent care credit'},
    'A07180':     {'readable_name': 'child_and_dependent_care_credit_amount', 'description': 'Child and dependent care credit amount'},
    'N07230':     {'readable_name': 'num_returns_with_nonrefundable_education_credit', 'description': 'Number of returns with nonrefundable education credit'},
    'A07230':     {'readable_name': 'nonrefundable_education_credit_amount', 'description': 'Nonrefundable education credit amount'},
    'N07240':     {'readable_name': 'num_returns_with_retirement_savings_contribution_credit', 'description': 'Number of returns with retirement savings contribution credit'},
    'A07240':     {'readable_name': 'retirement_savings_contribution_credit_amount', 'description': 'Retirement savings contribution credit amount'},
    'N07225':     {'readable_name': 'num_returns_with_child_and_other_dependent_credit', 'description': 'Number of returns with child and other dependent credit'},
    'A07225':     {'readable_name': 'child_and_other_dependent_credit_amount', 'description': 'Child and other dependent credit amount'},
    'N07260':     {'readable_name': 'num_returns_with_residential_energy_tax_credit', 'description': 'Number of returns with residential energy tax credit'},
    'A07260':     {'readable_name': 'residential_energy_tax_credit_amount', 'description': 'Residential energy tax credit amount'},
    'N09400':     {'readable_name': 'num_returns_with_self_employment_tax' , 'description': 'Number of returns with self-employment tax'},
    'A09400':     {'readable_name': 'self_employment_tax_amount', 'description': 'Self-employment tax amount'},
    'N85770':     {'readable_name': 'num_returns_with_total_premium_tax_credit', 'description': 'Number of returns with total premium tax credit'},
    'A85770':     {'readable_name': 'total_premium_tax_credit_amount', 'description': 'Total premium tax credit amount'},
    'N85775':     {'readable_name': 'num_returns_with_advance_premium_tax_credit', 'description': 'Number of returns with advance premium tax credit'},
    'A85775':     {'readable_name': 'advance_premium_tax_credit_amount', 'description': 'Advance premium tax credit amount'},
    'N10600':     {'readable_name': 'num_returns_with_total_tax_payments', 'description': 'Number of returns with total tax payments'},
    'A10600':     {'readable_name': 'total_tax_payments_amount', 'description': 'Total tax payments amount'},
    'N59660':     {'readable_name': 'num_returns_with_earned_income_credit', 'description': 'Number of returns with earned income credit'},
    'A59660':     {'readable_name': 'earned_income_credit_amount', 'description': 'Earned income credit amount'},
    'N59720':     {'readable_name': 'num_returns_with_excess_earned_income_credit', 'description': 'Number of returns with excess earned income credit (refundable)'},
    'A59720':     {'readable_name': 'excess_earned_income_credit_amount', 'description': 'Excess earned income credit (refundable) amount'},
    'N11070':     {'readable_name': 'num_returns_with_additional_child_tax_credit', 'description': 'Number of returns with additional child tax credit'},
    'A11070':     {'readable_name': 'additional_child_tax_credit_amount', 'description': 'Additional child tax credit amount'},
    'N10960':     {'readable_name': 'num_returns_with_refundable_education_credit', 'description': 'Number of returns with refundable education credit'},
    'A10960':     {'readable_name': 'refundable_education_credit_amount', 'description': 'Refundable education credit amount'},
    'N11560':     {'readable_name': 'num_returns_with_net_premium_tax_credit', 'description': 'Number of returns with net premium tax credit'},
    'A11560':     {'readable_name': 'net_premium_tax_credit_amount', 'description': 'Net premium tax credit amount'},
    'N11450':     {'readable_name': 'num_returns_with_qualified_sick_and_family_leave_credit', 'description': 'Number of returns with qualified sick and family leave credit'},
    'A11450':     {'readable_name': 'qualified_sick_and_family_leave_credit_amount', 'description': 'Qualified sick and family leave credit amount'},
    'N10970':     {'readable_name': 'num_returns_with_recovery_rebate_credit', 'description': 'Number of returns with recovery rebate credit'},
    'A10970':     {'readable_name': 'recovery_rebate_credit_amount', 'description': 'Recovery rebate credit amount'},
    'N10971':     {'readable_name': 'num_returns_with_economic_impact_payment_first_round', 'description': 'Number of returns with economic impact payment first round'},
    'A10971':     {'readable_name': 'economic_impact_payment_first_round_amount', 'description': 'Economic impact payment first round amount'},
    'N10973':     {'readable_name': 'num_returns_with_economic_impact_payment_second_round', 'description': 'Number of returns with economic impact payment second round'},
    'A10973':     {'readable_name': 'economic_impact_payment_second_round_amount', 'description': 'Economic impact payment second round amount'},
    'N06500':     {'readable_name': 'num_returns_with_income_tax_after_credits', 'description': 'Number of returns with income tax after credits'},
    'A06500':     {'readable_name': 'income_tax_after_credits_amount', 'description': 'Income tax after credits amount'},
    'N10300':     {'readable_name': 'num_returns_with_tax_liability', 'description': 'Number of returns with tax liability'},
    'A10300':     {'readable_name': 'total_tax_liability_amount', 'description': 'Total tax liability amount'},
    'N85530':     {'readable_name': 'num_returns_with_additional_medicare_tax', 'description': 'Number of returns with additional Medicare tax'},
    'A85530':     {'readable_name': 'additional_medicare_tax_amount', 'description': 'Additional Medicare tax amount'},
    'N85300':     {'readable_name': 'num_returns_with_net_investment_income_tax', 'description': 'Number of returns with net investment income tax'},
    'A85300':     {'readable_name': 'net_investment_income_tax_amount', 'description': 'Net investment income tax amount'},
    'N11901':     {'readable_name': 'num_returns_with_tax_due_at_time_of_filing', 'description': 'Number of returns with tax due at time of filing'},
    'A11901':     {'readable_name': 'tax_due_at_time_of_filing_amount', 'description': 'Tax due at time of filing amount'},
    'N11900':     {'readable_name': 'num_returns_with_total_overpayments', 'description': 'Number of returns with total overpayments'},
    'A11900':     {'readable_name': 'total_overpayments_amount', 'description': 'Total overpayments amount'},
    'N11902':     {'readable_name': 'num_returns_with_overpayments_refunded', 'description': 'Number of returns with overpayments refunded'},
    'A11902':     {'readable_name': 'overpayments_refunded_amount', 'description': 'Overpayments refunded amount'},
    'N12000':     {'readable_name': 'num_returns_with_credit_to_next_year_estimated_tax', 'description': 'Number of returns with credit to next year’s estimated tax'},
    'A12000':     {'readable_name': 'credited_to_next_year_estimated_tax_amount', 'description': 'Credited to next year’s estimated tax amount'}
}



In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('src/IRS/2023-05-19_14.11.58-20zpallnoagi.csv')

# Get the column names as a list
column_names = df.columns.tolist()

# Format and print the column names in a list form
formatted_list = '[' + ',\n '.join(column_names) + ']'
print(formatted_list)


In [ ]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('src/IRS/2023-05-19_14.11.58-20zpallnoagi.csv')

# Rename the columns using the field_mapping dictionary
df.rename(columns={k: v['readable_name'] for k, v in field_mapping.items()}, inplace=True)

# Modify the column names to match the desired format (lowercase with underscores)
df.columns = df.columns.str.lower().str.replace(' ', '_')


# Get the column names as a list
# column_names = df.columns.tolist()

# Format and print the column names in a list form
# formatted_list = '[' + ',\n '.join(column_names) + ']'
# print(formatted_list)


# Display the resulting DataFrame
print(df)


In [1]:
import pandas as pd
import json
import os
import sys
current_dir = os.path.dirname(os.path.abspath(__file__))  # Get the current script's directory
parent_dir = os.path.dirname(current_dir)  # Get the parent directory
sys.path.insert(0, parent_dir)  # Add the parent directory to the beginning of the system path

from helpers.clickhouse_operations import ClickHouseOperations

# read in the input CSV file to a Pandas DataFrame
# df = pd.read_csv('input_data.csv')

db = ClickHouseOperations()

query = """
    SELECT
        zc.`type`,
        zc.coordinates,
        irs.zip_code,
        irs.total_medical_and_dental_expense_deduction_amount
    FROM
        irs_soi_stats_2020_noagi irs
        INNER JOIN zip_codes zc 
            ON irs.zip_code = zc.zip
"""

df = db.query_df(query)

# create a new empty list to store the GeoJSON features
features = []

# loop through each row in the DataFrame
for index, row in df.iterrows():
    # get the type and coordinates columns
    geom_type = row['type']
    coordinates = json.loads(row['coordinates'])
    
    # create a new dictionary for the feature
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': geom_type,
            'coordinates': coordinates
        },
        'properties': {
            'zip_code': row['zip_code'],
            'total_medical_and_dental_expense_deduction_amount': row['total_medical_and_dental_expense_deduction_amount']
        }
    }
    
    # add the feature to the list
    features.append(feature)

# create the final GeoJSON dictionary with the features list
geojson_dict = {
    'type': 'FeatureCollection',
    'features': features
}

# output the GeoJSON to a file
with open('output_geojson.geojson', 'w') as f:
    json.dump(geojson_dict, f)

NameError: name '__file__' is not defined